In [52]:
import pandas as pd 
import matplotlib as pyplot
from pandasql import sqldf

In [53]:
df = pd.read_csv("../rawdata/hololivecomments.csv")
del df['subreddit.1']

In [54]:
df = df.dropna(axis=0, how="any", subset=None, inplace=False)

In [55]:
# Sort data berdasarkan waktu posting dan mengubah epoch value menjadi datetime UTC+7

df.sort_values(by = ["created_utc"], inplace = True, ascending = True)
df['created_utc'] = pd.DatetimeIndex(pd.to_datetime(df['created_utc'], unit='s')).tz_localize('UTC').tz_convert('Asia/Jakarta')

In [56]:
display(df.head(5))
display(df.tail(5))
display(df.info())

,subreddit,id,submission_id,created_utc,author,score,body,parent_id,permalink
406059,Hololive,ff4gful,erj4fp,2020-01-21 06:30:06+07:00,raystrife96,4.0,"I'm not up-to-date with Rushia, but I think th...",t3_erj4fp,/r/Hololive/comments/erj4fp/i_have_some_questi...
406061,Hololive,ff6nd9s,erj4fp,2020-01-22 01:21:24+07:00,raystrife96,2.0,"Yeah, if she changes to her default outfit, th...",t1_ff5k2n9,/r/Hololive/comments/erj4fp/i_have_some_questi...
406050,Hololive,ffhj7ub,etl9ue,2020-01-25 14:35:10+07:00,raystrife96,16.0,Okayu and Korone form a great pair too for cut...,t3_etl9ue,/r/Hololive/comments/etl9ue/what_pair_do_you_t...
406052,Hololive,ffhy6m7,etl9ue,2020-01-25 20:29:34+07:00,JustAnotherPep,3.0,"I haven't watched any of their stuff yet, I gu...",t1_ffhj7ub,/r/Hololive/comments/etl9ue/what_pair_do_you_t...
406051,Hololive,ffi49xe,etl9ue,2020-01-25 22:04:17+07:00,GuiMr27,3.0,Hmmmmmm... maybe FAMS? Idk,t3_etl9ue,/r/Hololive/comments/etl9ue/what_pair_do_you_t...


,subreddit,id,submission_id,created_utc,author,score,body,parent_id,permalink
311133,Hololive,j9hbarx,i4oabc,2023-02-22 06:43:31+07:00,greenninjamaster02,1.0,NO DONT DO IT DONT BECOME HISOKA,t3_i4oabc,/r/Hololive/comments/i4oabc/she_hasnt_even_deb...
1639695,Hololive,j9iz5e0,vgg8gb,2023-02-22 15:36:18+07:00,knktzvra,1.0,What about this?\n\n\-Kanata used to eat grass...,t3_vgg8gb,/r/Hololive/comments/vgg8gb/what_are_some_fun_...
1639921,Hololive,j9j7588,vgg8gb,2023-02-22 17:28:50+07:00,StarPK117,2.0,Thanks but how did you found this 8 months later?,t1_j9iz5e0,/r/Hololive/comments/vgg8gb/what_are_some_fun_...
1640097,Hololive,j9ljnpn,vgg8gb,2023-02-23 04:12:39+07:00,knktzvra,1.0,"I was searching about Korone, since I felt she...",t1_j9j7588,/r/Hololive/comments/vgg8gb/what_are_some_fun_...
2081782,Hololive,j9pnmpa,pmt29s,2023-02-24 01:01:25+07:00,ninjasonic102,1.0,Did you ever end up making a new one? I wanted...,t1_iclrptc,/r/Hololive/comments/pmt29s/violet_takodachi_g...
1724188,Hololive,j9sxb0r,whonbx,2023-02-24 16:21:22+07:00,CombineElite3650,1.0,Laplus is still Keter because that lock probab...,t1_ijcfgsc,/r/Hololive/comments/whonbx/hololive_members_r...
1724185,Hololive,j9sxesu,whonbx,2023-02-24 16:22:51+07:00,CombineElite3650,1.0,Nah she is 343s daughter and Jebus is her half...,t1_ij7knpk,/r/Hololive/comments/whonbx/hololive_members_r...
1724181,Hololive,j9sxkgr,whonbx,2023-02-24 16:25:03+07:00,CombineElite3650,1.0,That's not good.,t1_ij7td8x,/r/Hololive/comments/whonbx/hololive_members_r...
2383500,Hololive,j9w1wxe,zg3uzm,2023-02-25 06:35:42+07:00,DarklyDreamingEva,1.0,I hear the people at Counterside have brought ...,t3_zg3uzm,/r/Hololive/comments/zg3uzm/psa_the_countersid...
2308571,Hololive,j9xwi9v,xosjon,2023-02-25 17:30:10+07:00,3Lesate3,2.0,Like it very much. \nThanks for this :),t3_xosjon,/r/Hololive/comments/xosjon/faunas_death_road_...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2285265 entries, 406059 to 406048
Data columns (total 9 columns):
 #   Column         Dtype                       
---  ------         -----                       
 0   subreddit      object                      
 1   id             object                      
 2   submission_id  object                      
 3   created_utc    datetime64[ns, Asia/Jakarta]
 4   author         object                      
 5   score          float64                     
 6   body           object                      
 7   parent_id      object                      
 8   permalink      object                      
dtypes: datetime64[ns, Asia/Jakarta](1), float64(1), object(7)
memory usage: 174.4+ MB


None